 # Introduction

Dedicated API: https://api-datalab.coderslab.com/api/v2

Documantation: https://api-datalab.coderslab.com/v2/docs/

Endpoints:
 - `airport` - airports data
 - `weather` - weather at the airport per day
 - `aircraft` - airplanes data
 - `flights` - departures from a given airport per day

Airport ID list: `airports.csv`

Token: `iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk`

 ## Notebook config

In [1]:
import requests
import pandas as pd
import datetime
import time

In [2]:
url = 'https://api-datalab.coderslab.com/api/v2'
token = 'iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk'
path = '../data/airports.csv'

## Getting airports ids from a file and testing a single request

In [3]:
df = pd.read_csv(path)

In [4]:
airportId = 11638
response = requests.get(f'{url}/airport/{airportId}', headers={'Authorization': token})
data = response.json()
#print(type(data))
print(response.json())
columns = ['ORIGIN_AIRPORT_ID', 'DISPLAY_AIRPORT_NAME', 'ORIGIN_CITY_NAME', 'NAME']
df1 = pd.DataFrame([data], columns=columns)

{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}


 # Downloading `Airport`

In [5]:
columns = ['ORIGIN_AIRPORT_ID', 'DISPLAY_AIRPORT_NAME', 'ORIGIN_CITY_NAME', 'NAME']
airport = []
for id in df['origin_airport_id']:
    airportId = id
    response = requests.get(f'{url}/airport/{airportId}', headers={'Authorization': token})
    if response.ok == True:
        data = response.json()
        airport.append(data)

airport_df = pd.DataFrame(airport, columns=columns)

In [6]:
airport_df

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,11638,Fresno Air Terminal,"Fresno, CA","FRESNO YOSEMITE INTERNATIONAL, CA US"
1,13342,General Mitchell Field,"Milwaukee, WI","MILWAUKEE MITCHELL AIRPORT, WI US"
2,13244,Memphis International,"Memphis, TN","MEMPHIS INTERNATIONAL AIRPORT, TN US"
3,15096,Syracuse Hancock International,"Syracuse, NY","SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US"
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
...,...,...,...,...
92,13198,Kansas City International,"Kansas City, MO","KANSAS CITY INTERNATIONAL AIRPORT, MO US"
93,10423,Austin - Bergstrom International,"Austin, TX","AUSTIN BERGSTROM INTERNATIONAL AIRPORT, TX US"
94,15370,Tulsa International,"Tulsa, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
95,13303,Miami International,"Miami, FL","MIAMI INTERNATIONAL AIRPORT, FL US"


 ## Saving data to a .csv file

In [8]:
airport_df.to_csv('airport_list.csv', mode='w+', encoding="UTF-8", index=False, sep=',')

 # Downloading `Weather`

- Start date: `2019-01-01`
- End date: `2020-03-31`

In [9]:
date = datetime.date(2019, 1, 1)
date = date.strftime('%Y-%m')
response = requests.get(f'{url}/airportWeather?date={date}', headers={'Authorization': token})
data = response.json()
data_df = pd.json_normalize(data)

In [10]:
start_date = datetime.date(2019, 1, 1)
end_date = datetime.date(2020, 3, 31)

date_list = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m')

In [11]:
weather = []

for date in date_list:
    response = requests.get(f'{url}/airportWeather?date={date}', headers={'Authorization': token})
    if response.ok == True:
        data = response.json()
        weather.append(data)

flat_weather = [item for sublist in weather for item in sublist] 

airport_weather_df = pd.DataFrame(flat_weather)

 ## Saving data to a .csv file

In [10]:
airport_weather_df.to_csv('../data/raw/airport_weather.csv', mode='w+', encoding="UTF-8", index=False, sep=',')

 # Downloading `Aircraft`

In [13]:
response = requests.get(f'{url}/aircraft', headers={'Authorization': token})
data = response.json()
aircraft_df = pd.DataFrame(data)

 ## Saving data to a .csv file

In [15]:
aircraft_df.to_csv('../data/raw/aircraft.csv', mode='w+', encoding="UTF-8", index=False, sep=',')

 # Downloading `Flight`

- Start date: `2019-01-01`
- End date: `2020-03-31`
- print() checkpoints added for clarity

In [16]:
airport_df = pd.read_csv('../data/airports.csv')

In [17]:
flight_data = []
for airport_id in airport_df['origin_airport_id']:
    for date in date_list:
        response = requests.get(f'{url}/flight?airportId={airport_id}&date={date}', headers={'Authorization': f'{token}'})
        if response.status_code == 200:
            f_data = response.json()
            flight_data.append(f_data)
            print(f' | Downloaded raw for airport {airport_id}, {date}')
        elif response.status_code == 204:
            print(f' | Error 204 for airport {airport_id}, {date}')
            pass

 | Error 204 for airport 10874, 2019-01
 | Error 204 for airport 10874, 2019-02
 | Error 204 for airport 10874, 2019-03
 | Error 204 for airport 10874, 2019-04
 | Error 204 for airport 10874, 2019-05
 | Error 204 for airport 10874, 2019-06
 | Error 204 for airport 10874, 2019-07
 | Error 204 for airport 10874, 2019-08
 | Error 204 for airport 10874, 2019-09
 | Error 204 for airport 10874, 2019-10
 | Error 204 for airport 10874, 2019-11
 | Error 204 for airport 10874, 2019-12
 | Error 204 for airport 10874, 2020-01
 | Error 204 for airport 10874, 2020-02
 | Error 204 for airport 10874, 2020-03
 | Error 204 for airport 11233, 2019-01
 | Error 204 for airport 11233, 2019-02
 | Error 204 for airport 11233, 2019-03
 | Error 204 for airport 11233, 2019-04
 | Error 204 for airport 11233, 2019-05
 | Error 204 for airport 11233, 2019-06
 | Error 204 for airport 11233, 2019-07
 | Error 204 for airport 11233, 2019-08
 | Error 204 for airport 11233, 2019-09
 | Error 204 for airport 11233, 2019-10


 | Error 204 for airport 12280, 2019-11
 | Error 204 for airport 12280, 2019-12
 | Error 204 for airport 12280, 2020-01
 | Error 204 for airport 12280, 2020-02
 | Error 204 for airport 12280, 2020-03
 | Error 204 for airport 15096, 2019-01
 | Error 204 for airport 15096, 2019-02
 | Error 204 for airport 15096, 2019-03
 | Error 204 for airport 15096, 2019-04
 | Error 204 for airport 15096, 2019-05
 | Error 204 for airport 15096, 2019-06
 | Error 204 for airport 15096, 2019-07
 | Error 204 for airport 15096, 2019-08
 | Error 204 for airport 15096, 2019-09
 | Error 204 for airport 15096, 2019-10
 | Error 204 for airport 15096, 2019-11
 | Error 204 for airport 15096, 2019-12
 | Error 204 for airport 15096, 2020-01
 | Error 204 for airport 15096, 2020-02
 | Error 204 for airport 15096, 2020-03
 | Error 204 for airport 11641, 2019-01
 | Error 204 for airport 11641, 2019-02
 | Error 204 for airport 11641, 2019-03
 | Error 204 for airport 11641, 2019-04
 | Error 204 for airport 11641, 2019-05


 | Error 204 for airport 10140, 2019-04
 | Error 204 for airport 10140, 2019-05
 | Error 204 for airport 10140, 2019-06
 | Error 204 for airport 10140, 2019-07
 | Error 204 for airport 10140, 2019-08
 | Error 204 for airport 10140, 2019-09
 | Error 204 for airport 10140, 2019-10
 | Error 204 for airport 10140, 2019-11
 | Error 204 for airport 10140, 2019-12
 | Error 204 for airport 10140, 2020-01
 | Error 204 for airport 10140, 2020-02
 | Error 204 for airport 10140, 2020-03
 | Error 204 for airport 12389, 2019-01
 | Error 204 for airport 12389, 2019-02
 | Error 204 for airport 12389, 2019-03
 | Error 204 for airport 12389, 2019-04
 | Error 204 for airport 12389, 2019-05
 | Error 204 for airport 12389, 2019-06
 | Error 204 for airport 12389, 2019-07
 | Error 204 for airport 12389, 2019-08
 | Error 204 for airport 12389, 2019-09
 | Error 204 for airport 12389, 2019-10
 | Error 204 for airport 12389, 2019-11
 | Error 204 for airport 12389, 2019-12
 | Error 204 for airport 12389, 2020-01


 | Error 204 for airport 15841, 2020-02
 | Error 204 for airport 15841, 2020-03
 | Downloaded raw for airport 14831, 2019-01
 | Downloaded raw for airport 14831, 2019-02
 | Downloaded raw for airport 14831, 2019-03
 | Downloaded raw for airport 14831, 2019-04
 | Downloaded raw for airport 14831, 2019-05
 | Downloaded raw for airport 14831, 2019-06
 | Downloaded raw for airport 14831, 2019-07
 | Downloaded raw for airport 14831, 2019-08
 | Downloaded raw for airport 14831, 2019-09
 | Downloaded raw for airport 14831, 2019-10
 | Downloaded raw for airport 14831, 2019-11
 | Downloaded raw for airport 14831, 2019-12
 | Downloaded raw for airport 14831, 2020-01
 | Downloaded raw for airport 14831, 2020-02
 | Downloaded raw for airport 14831, 2020-03
 | Error 204 for airport 12898, 2019-01
 | Error 204 for airport 12898, 2019-02
 | Error 204 for airport 12898, 2019-03
 | Error 204 for airport 12898, 2019-04
 | Error 204 for airport 12898, 2019-05
 | Error 204 for airport 12898, 2019-06
 | Er

 | Error 204 for airport 11109, 2019-06
 | Error 204 for airport 11109, 2019-07
 | Error 204 for airport 11109, 2019-08
 | Error 204 for airport 11109, 2019-09
 | Error 204 for airport 11109, 2019-10
 | Error 204 for airport 11109, 2019-11
 | Error 204 for airport 11109, 2019-12
 | Error 204 for airport 11109, 2020-01
 | Error 204 for airport 11109, 2020-02
 | Error 204 for airport 11109, 2020-03
 | Error 204 for airport 13459, 2019-01
 | Error 204 for airport 13459, 2019-02
 | Error 204 for airport 13459, 2019-03
 | Error 204 for airport 13459, 2019-04
 | Error 204 for airport 13459, 2019-05
 | Error 204 for airport 13459, 2019-06
 | Error 204 for airport 13459, 2019-07
 | Error 204 for airport 13459, 2019-08
 | Error 204 for airport 13459, 2019-09
 | Error 204 for airport 13459, 2019-10
 | Error 204 for airport 13459, 2019-11
 | Error 204 for airport 13459, 2019-12
 | Error 204 for airport 13459, 2020-01
 | Error 204 for airport 13459, 2020-02
 | Error 204 for airport 13459, 2020-03


...

 | Error 204 for airport 12197, 2019-01
 | Error 204 for airport 12197, 2019-02
 | Error 204 for airport 12197, 2019-03
 | Error 204 for airport 12197, 2019-04
 | Error 204 for airport 12197, 2019-05
 | Error 204 for airport 12197, 2019-06
 | Error 204 for airport 12197, 2019-07
 | Error 204 for airport 12197, 2019-08
 | Error 204 for airport 12197, 2019-09
 | Error 204 for airport 12197, 2019-10
 | Error 204 for airport 12197, 2019-11
 | Error 204 for airport 12197, 2019-12
 | Error 204 for airport 12197, 2020-01
 | Error 204 for airport 12197, 2020-02
 | Error 204 for airport 12197, 2020-03
 | Error 204 for airport 14696, 2019-01
 | Error 204 for airport 14696, 2019-02
 | Error 204 for airport 14696, 2019-03
 | Error 204 for airport 14696, 2019-04
 | Error 204 for airport 14696, 2019-05
 | Error 204 for airport 14696, 2019-06
 | Error 204 for airport 14696, 2019-07
 | Error 204 for airport 14696, 2019-08
 | Error 204 for airport 14696, 2019-09
 | Error 204 for airport 14696, 2019-10


 | Error 204 for airport 14098, 2019-11
 | Error 204 for airport 14098, 2019-12
 | Error 204 for airport 14098, 2020-01
 | Error 204 for airport 14098, 2020-02
 | Error 204 for airport 14098, 2020-03
 | Downloaded raw for airport 12982, 2019-01
 | Downloaded raw for airport 12982, 2019-02
 | Downloaded raw for airport 12982, 2019-03
 | Downloaded raw for airport 12982, 2019-04
 | Downloaded raw for airport 12982, 2019-05
 | Downloaded raw for airport 12982, 2019-06
 | Downloaded raw for airport 12982, 2019-07
 | Downloaded raw for airport 12982, 2019-08
 | Downloaded raw for airport 12982, 2019-09
 | Downloaded raw for airport 12982, 2019-10
 | Downloaded raw for airport 12982, 2019-11
 | Downloaded raw for airport 12982, 2019-12
 | Downloaded raw for airport 12982, 2020-01
 | Downloaded raw for airport 12982, 2020-02
 | Downloaded raw for airport 12982, 2020-03
 | Error 204 for airport 14512, 2019-01
 | Error 204 for airport 14512, 2019-02
 | Error 204 for airport 14512, 2019-03
 | Er

 | Error 204 for airport 12896, 2020-02
 | Error 204 for airport 12896, 2020-03
 | Error 204 for airport 10279, 2019-01
 | Error 204 for airport 10279, 2019-02
 | Error 204 for airport 10279, 2019-03
 | Error 204 for airport 10279, 2019-04
 | Error 204 for airport 10279, 2019-05
 | Error 204 for airport 10279, 2019-06
 | Error 204 for airport 10279, 2019-07
 | Error 204 for airport 10279, 2019-08
 | Error 204 for airport 10279, 2019-09
 | Error 204 for airport 10279, 2019-10
 | Error 204 for airport 10279, 2019-11
 | Error 204 for airport 10279, 2019-12
 | Error 204 for airport 10279, 2020-01
 | Error 204 for airport 10279, 2020-02
 | Error 204 for airport 10279, 2020-03
 | Error 204 for airport 11778, 2019-01
 | Error 204 for airport 11778, 2019-02
 | Error 204 for airport 11778, 2019-03
 | Error 204 for airport 11778, 2019-04
 | Error 204 for airport 11778, 2019-05
 | Error 204 for airport 11778, 2019-06
 | Error 204 for airport 11778, 2019-07
 | Error 204 for airport 11778, 2019-08


 | Error 204 for airport 15624, 2019-07
 | Error 204 for airport 15624, 2019-08
 | Error 204 for airport 15624, 2019-09
 | Error 204 for airport 15624, 2019-10
 | Error 204 for airport 15624, 2019-11
 | Error 204 for airport 15624, 2019-12
 | Error 204 for airport 15624, 2020-01
 | Error 204 for airport 15624, 2020-02
 | Error 204 for airport 15624, 2020-03
 | Error 204 for airport 10994, 2019-01
 | Error 204 for airport 10994, 2019-02
 | Error 204 for airport 10994, 2019-03
 | Error 204 for airport 10994, 2019-04
 | Error 204 for airport 10994, 2019-05
 | Error 204 for airport 10994, 2019-06
 | Error 204 for airport 10994, 2019-07
 | Error 204 for airport 10994, 2019-08
 | Error 204 for airport 10994, 2019-09
 | Error 204 for airport 10994, 2019-10
 | Error 204 for airport 10994, 2019-11
 | Error 204 for airport 10994, 2019-12
 | Error 204 for airport 10994, 2020-01
 | Error 204 for airport 10994, 2020-02
 | Error 204 for airport 10994, 2020-03
 | Error 204 for airport 10409, 2019-01


 | Error 204 for airport 13198, 2019-02
 | Error 204 for airport 13198, 2019-03
 | Error 204 for airport 13198, 2019-04
 | Error 204 for airport 13198, 2019-05
 | Error 204 for airport 13198, 2019-06
 | Error 204 for airport 13198, 2019-07
 | Error 204 for airport 13198, 2019-08
 | Error 204 for airport 13198, 2019-09
 | Error 204 for airport 13198, 2019-10
 | Error 204 for airport 13198, 2019-11
 | Error 204 for airport 13198, 2019-12
 | Error 204 for airport 13198, 2020-01
 | Error 204 for airport 13198, 2020-02
 | Error 204 for airport 13198, 2020-03
 | Error 204 for airport 10423, 2019-01
 | Error 204 for airport 10423, 2019-02
 | Error 204 for airport 10423, 2019-03
 | Error 204 for airport 10423, 2019-04
 | Error 204 for airport 10423, 2019-05
 | Error 204 for airport 10423, 2019-06
 | Error 204 for airport 10423, 2019-07
 | Error 204 for airport 10423, 2019-08
 | Error 204 for airport 10423, 2019-09
 | Error 204 for airport 10423, 2019-10
 | Error 204 for airport 10423, 2019-11


In [18]:
flat_data = [item for sublist in flight_data for item in sublist] 
flat_data_df = pd.DataFrame(flat_data)

In [19]:
flat_data_df.shape

(1386120, 27)

 ## Saving data to .csv file

In [10]:
flat_data_df.to_csv('../data/raw/flight.csv', mode='w+', encoding="UTF-8", index=False, sep=',')